[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/12SjJRzRrqsP10dI5jmWtcL8l-BZQfOXG/view?usp=sharing)

Since the model is devoleped using tf version 1 we are installing the version tf.1.xx

In [ ]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5 MB 8.0 kB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
     |████████████████████████████████| 3.8 MB 33.7 MB/s 
     |████████████████████████████████| 503 kB 39.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=7b1d7dac5d1b482bc64f9b8dcc2f582b8ce8a99ca977b6df35c0cf44da73c856
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/NVIDIA/OpenSeq2Seq.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !pip uninstall -y -q pymc3
  !pip install --upgrade joblib
  !pip install -q youtube-dl librosa python_speech_features sentencepiece
  
  # create eval config
  !cp {project_name}/example_configs/speech2text/jasper10x5_LibriSpeech_nvgrad.py {project_name}/conf.py
  !sed -i -e 's/\/data\/librispeech\/librivox-test-clean/test/' {project_name}/conf.py
  !echo 'backend = "librosa"' >> {project_name}/conf.py 
  # !echo "wav_filename, wav_filesize, transcript" > {project_name}/test.csv
  # !echo "test.wav, UNUSED, UNUSED" >> {project_name}/test.csv
  
import sys
sys.path.append(project_name)

     |████████████████████████████████| 1.9 MB 3.8 MB/s 
     |████████████████████████████████| 1.2 MB 35.0 MB/s 


In [ ]:
#!gdown --id 1HH23gWovcBwRQTh0kOwYIl55I-P0LtIY
# !gdown --id 1En86LcxRw701XIXOy31wHx4NTUTluYzY
# !gdown --id 1oEVk04zVpQkvW9YdgRQ9k7tuMPvbEbLq #For Clean data
!gdown --id 11r0C6IkSvZIAheHFnD_Ts72IxOAN96i7 #For other data


Downloading...
From: https://drive.google.com/uc?id=1oEVk04zVpQkvW9YdgRQ9k7tuMPvbEbLq
To: /content/FullData.zip
100% 508M/508M [00:04<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=11r0C6IkSvZIAheHFnD_Ts72IxOAN96i7
To: /content/OtherFullData.zip
100% 491M/491M [00:04<00:00, 101MB/s]


In [ ]:
!unzip /content/OtherFullData.zip

In [ ]:
import pandas as pd

In [ ]:
df_testu = pd.read_csv("/content/OtherFullData/test.csv")

In [ ]:
df_testu

,wav_filename,wav_filesize,transcript
0,file_00.wav,UNUSED,UNUSED
1,file_01.wav,UNUSED,UNUSED
2,file_010.wav,UNUSED,UNUSED
3,file_0100.wav,UNUSED,UNUSED
4,file_01000.wav,UNUSED,UNUSED
...,...,...,...
2934,file_0995.wav,UNUSED,UNUSED
2935,file_0996.wav,UNUSED,UNUSED
2936,file_0997.wav,UNUSED,UNUSED
2937,file_0998.wav,UNUSED,UNUSED


In [ ]:
for i in range(df_testu.shape[0]):
  path = "OtherFullData/audio/"
  name = df_testu["wav_filename"][i]
  new_path = os.path.join(path, name)
  df_testu["wav_filename"][i] = new_path

In [ ]:
df_testu

,wav_filename,wav_filesize,transcript
0,OtherFullData/audio/file_00.wav,UNUSED,UNUSED
1,OtherFullData/audio/file_01.wav,UNUSED,UNUSED
2,OtherFullData/audio/file_010.wav,UNUSED,UNUSED
3,OtherFullData/audio/file_0100.wav,UNUSED,UNUSED
4,OtherFullData/audio/file_01000.wav,UNUSED,UNUSED
...,...,...,...
2934,OtherFullData/audio/file_0995.wav,UNUSED,UNUSED
2935,OtherFullData/audio/file_0996.wav,UNUSED,UNUSED
2936,OtherFullData/audio/file_0997.wav,UNUSED,UNUSED
2937,OtherFullData/audio/file_0998.wav,UNUSED,UNUSED


In [ ]:
df_testu.to_csv("/content/OpenSeq2Seq/test.csv")

In [ ]:
!cp -r /content/OtherFullData /content/OpenSeq2Seq

## Download pre-trained Model

In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
if not exists(join(project_name, 'w2l_log_folder')):
  download_from_google_drive('1gzGT8HoVNKY1i5HNQTKaSoCu7JHV4siR', 'jasper_10x5_dr_sp_nvgrad.zip')
  !unzip jasper_10x5_dr_sp_nvgrad.zip
  !mv checkpoint {project_name}/jasper_log_folder

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1406      0 --:--:-- --:--:-- --:--:--  1406
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2960M  100 2960M    0     0  55.4M      0  0:00:53  0:00:53 --:--:-- 35.6M
Archive:  jasper_10x5_dr_sp_nvgrad.zip
   creating: checkpoint/
  inflating: checkpoint/checkpoint   
  inflating: checkpoint/model.ckpt-439200.data-00000-of-00001  
  inflating: checkpoint/model.ckpt-439200.index  
  inflating: checkpoint/model.ckpt-439200.meta  


In [ ]:
!cd {project_name} && python run.py --config_file conf.py --mode=infer --infer_output_file=NoLMOtherFull.txt --use_horovod=False --num_gpus=1 --batch_size_per_gpu 1












The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



*** Restoring from the latest checkpoint
*** Loading model from jasper_log_folder/model.ckpt-439200
*** Inference config:
{'batch_size_per_gpu': 1,
 'data_layer': <class 'open_seq2seq.data.speech2text.speech2text.Speech2TextDataLayer'>,
 'data_layer_params': {'backend': 'librosa',
                       'dataset_files': ['test.csv'],
                       'dither': 1e-05,
                       'input_type': 'logfbank',
                       'norm_per_feature': True,
                       'num_audio_features': 64,
                       'pad_to': 16,
                       'precompute_mel_basis': True,
          

In [ ]:
!cat '/content/OpenSeq2Seq/NoLMOtherFull.txt'

wav_filename,predicted_transcript
OtherFullData/audio/file_00.wav,ther's iron they say in all our blood and a groin or two perhaps is good but his he makes me harshly feel has got a little too much of steel anon
OtherFullData/audio/file_01.wav,margaret said mister hale as he returned from showing his guests downstairs i could not help watching your face with some anxiety when mister thornton made his confession of having been a shopboy
OtherFullData/audio/file_010.wav,his father speculated wildly failed and then killed himself because he could not bear the disgrace
OtherFullData/audio/file_0100.wav,the gut and the gallege being cut across between these ligages the stomach may be removed entire without spinning his contents
OtherFullData/audio/file_01000.wav,you are not like other men sar the madeleine
OtherFullData/audio/file_01001.wav,this offers the means but give me some information in the first place
OtherFullData/audio/file_01002.wav,how long is the coffin six feet
OtherFullData/a

In [ ]:
!pip install jiwer

     |████████████████████████████████| 50 kB 3.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149865 sha256=75d1cb5a799ba094b1693d3ad42804267171bbff7ccad075af2f69ad5fdc8606
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
file2 = open("/content/OpenSeq2Seq/NoLMOtherFull.txt","r+") 
file2.readline(0)
X2 = file2.readlines()
file2.close()

In [ ]:
NoLMPredictions = list(X2)

In [ ]:
len(NoLMPredictions)

2940

In [ ]:
import pandas as pd

In [ ]:
actual_text = pd.read_csv('/content/OpenSeq2Seq/OtherFullData/text.csv')

In [ ]:
from jiwer import wer

In [ ]:
NoLMPredictions[0:5]

['wav_filename,predicted_transcript\n',
 "OtherFullData/audio/file_00.wav,ther's iron they say in all our blood and a groin or two perhaps is good but his he makes me harshly feel has got a little too much of steel anon\n",
 'OtherFullData/audio/file_01.wav,margaret said mister hale as he returned from showing his guests downstairs i could not help watching your face with some anxiety when mister thornton made his confession of having been a shopboy\n',
 'OtherFullData/audio/file_010.wav,his father speculated wildly failed and then killed himself because he could not bear the disgrace\n',
 'OtherFullData/audio/file_0100.wav,the gut and the gallege being cut across between these ligages the stomach may be removed entire without spinning his contents\n']

In [ ]:
NoLMPredictions.remove("wav_filename,predicted_transcript\n")

In [ ]:
len(NoLMPredictions)

2939

In [ ]:
NoLMPredictions[0][len(NoLMPredictions[0].split(",")[0])+1:-1]

"ther's iron they say in all our blood and a groin or two perhaps is good but his he makes me harshly feel has got a little too much of steel anon"

In [ ]:
NoLMprediction_list = []
for i in range(len(NoLMPredictions)):
  temp = NoLMPredictions[i][len(NoLMPredictions[i].split(",")[0])+1:-1]
  NoLMprediction_list.append(temp)

In [ ]:
NoLMprediction_list

# Results for Other data

In [ ]:
NoLMwer_list = []
for i in range(len(NoLMPredictions)):
  indexu = int(os.path.basename(NoLMPredictions[i].split(",")[0])[6:-4])
  wer_temp2 = wer(actual_text['text'][indexu].lower(),NoLMprediction_list[i])
  NoLMwer_list.append(wer_temp2)

In [ ]:
NoLMwer_list

In [ ]:
Avg_wer = sum(NoLMwer_list) / len(NoLMwer_list)
Avg_wer

0.13477643793859198

In [ ]:
indexu = int(os.path.basename(NoLMPredictions[1437].split(",")[0])[6:-4])
actual_text['text'][indexu].lower()

'signed wilfrid pigeoncote'

In [ ]:
NoLMprediction_list[1437]

'signed wilfred peach and coat'

In [ ]:
count = 0
for i, j in enumerate(NoLMwer_list):
  if j >= 1:
    count = count+1
    print("At index: ",i, "Values is: ",j)
print(count)

At index:  509 Values is:  1.0
At index:  755 Values is:  1.0
At index:  1437 Values is:  1.3333333333333333
At index:  1979 Values is:  1.6666666666666667
At index:  2178 Values is:  1.3
At index:  2248 Values is:  1.125
At index:  2260 Values is:  1.1428571428571428
At index:  2304 Values is:  1.0
8


In [ ]:
count = 0
for i, j in enumerate(NoLMwer_list):
  if j >= 0.5:
    count = count+1
    print("At index: ",i, "Values is: ",j)
print(count)

At index:  47 Values is:  0.625
At index:  69 Values is:  0.6666666666666666
At index:  82 Values is:  0.6
At index:  127 Values is:  0.6
At index:  169 Values is:  0.5
At index:  174 Values is:  0.5
At index:  203 Values is:  0.5
At index:  212 Values is:  0.5
At index:  217 Values is:  0.75
At index:  218 Values is:  0.6666666666666666
At index:  242 Values is:  0.5
At index:  243 Values is:  0.6
At index:  245 Values is:  0.5
At index:  252 Values is:  0.8
At index:  309 Values is:  0.6
At index:  402 Values is:  0.5
At index:  404 Values is:  0.6
At index:  405 Values is:  0.5384615384615384
At index:  497 Values is:  0.5555555555555556
At index:  502 Values is:  0.7272727272727273
At index:  509 Values is:  1.0
At index:  523 Values is:  0.5555555555555556
At index:  538 Values is:  0.5454545454545454
At index:  566 Values is:  0.64
At index:  580 Values is:  0.6666666666666666
At index:  623 Values is:  0.5833333333333334
At index:  643 Values is:  0.5
At index:  646 Values is:  

# Results for Clean data

In [ ]:
NoLMwer_list = []
for i in range(len(NoLMPredictions)):
  indexu = int(os.path.basename(NoLMPredictions[i].split(",")[0])[6:-4])
  wer_temp2 = wer(actual_text['text'][indexu].lower(),NoLMprediction_list[i])
  NoLMwer_list.append(wer_temp2)

In [ ]:
NoLMwer_list

In [ ]:
Avg_wer = sum(NoLMwer_list) / len(NoLMwer_list)
Avg_wer

0.04387671045989565